# Iterated EKF (IEKF) vs EKF for Differential Drive

**Coursework:** Recursive Estimation Filters — Homework 1  
**Student:** Shahid Ahamed Hasib

This notebook mirrors the EKF notebook but adds an Iterated EKF (Gauss-Newton refinement on the measurement update) to compare performance. Both filters share the same simulation, and we sweep IMU and position noise to highlight differences.

Deliverables included here:
* Clean simulation of truth, IMU, and position measurements (100 Hz vs 1 Hz).
* EKF and IEKF implementations with explicit Jacobians.
* Trajectory plots that mark position measurement times on the ground-truth path, matching the assignment rubric.
* Noise experiments (low vs high IMU and position noise) plus RMSE comparisons.
* Commentary on when IEKF helps (or not) given linear position measurements.


In [ ]:
import math
from dataclasses import dataclass
from typing import List, Tuple

import matplotlib.pyplot as plt
import numpy as np

np.set_printoptions(precision=3, suppress=True)


## Shared simulation
Same motion model as the EKF notebook. IMU noise and position noise are configurable per scenario.


In [ ]:
@dataclass
class SimConfig:
    t_final: float = 40.0
    dt_imu: float = 0.01
    dt_pos: float = 1.0
    accel_std: float = 0.2
    omega_std: float = 0.03
    pos_std: float = 0.5
    seed: int = 11

@dataclass
class SimData:
    t: np.ndarray
    x_true: np.ndarray
    imu_accel: np.ndarray
    imu_omega: np.ndarray
    pos_meas: np.ndarray
    pos_times: np.ndarray
    pos_indices: np.ndarray


def simulate_truth_and_measurements(cfg: SimConfig) -> SimData:
    rng = np.random.default_rng(cfg.seed)
    t = np.arange(0.0, cfg.t_final + cfg.dt_imu, cfg.dt_imu)
    n = len(t)

    x_true = np.zeros((n, 4))
    x_true[0] = np.array([0.0, 0.0, 0.0, 0.5])
    accel_cmd = 0.25 * np.sin(0.3 * t) + 0.05
    omega_cmd = 0.2 * np.cos(0.25 * t)

    imu_accel = accel_cmd + rng.normal(0.0, cfg.accel_std, size=n)
    imu_omega = omega_cmd + rng.normal(0.0, cfg.omega_std, size=n)

    for k in range(1, n):
        dt = cfg.dt_imu
        x, y, th, v = x_true[k - 1]
        v = v + accel_cmd[k - 1] * dt
        th = th + omega_cmd[k - 1] * dt
        x = x + v * math.cos(th) * dt
        y = y + v * math.sin(th) * dt
        x_true[k] = np.array([x, y, th, v])

    pos_times = np.arange(0.0, cfg.t_final + 1e-6, cfg.dt_pos)
    pos_indices = (pos_times / cfg.dt_imu).astype(int)
    pos_truth = x_true[pos_indices][:, :2]
    pos_meas = pos_truth + rng.normal(0.0, cfg.pos_std, size=pos_truth.shape)
    return SimData(t, x_true, imu_accel, imu_omega, pos_meas, pos_times, pos_indices)


## Filter implementations
Position measurements are linear, so the IEKF only differs in how it iterates the measurement update (useful when the measurement model is nonlinear). We still show the IEKF mechanics for completeness.


In [ ]:
def motion_jacobian(x: np.ndarray, dt: float, accel_m: float, omega_m: float):
    _, _, th, v = x
    F = np.array([
        [1.0, 0.0, -v * math.sin(th) * dt, math.cos(th) * dt],
        [0.0, 1.0,  v * math.cos(th) * dt, math.sin(th) * dt],
        [0.0, 0.0, 1.0, 0.0],
        [0.0, 0.0, 0.0, 1.0],
    ])
    G = np.array([
        [0.0, 0.0],
        [0.0, 0.0],
        [0.0, dt],
        [dt, 0.0],
    ])
    return F, G


def propagate(x: np.ndarray, accel_m: float, omega_m: float, dt: float):
    xk, yk, thk, vk = x
    vk1 = vk + accel_m * dt
    thk1 = thk + omega_m * dt
    xk1 = xk + vk * math.cos(thk) * dt
    yk1 = yk + vk * math.sin(thk) * dt
    return np.array([xk1, yk1, thk1, vk1])


def ekf_filter(sim: SimData, cfg: SimConfig) -> np.ndarray:
    n = len(sim.t)
    x_est = np.zeros((n, 4))
    x = np.array([0.0, 0.0, 0.0, 0.5])
    P = np.diag([1.0, 1.0, (5 * math.pi / 180) ** 2, 1.0])
    R = np.diag([cfg.pos_std**2, cfg.pos_std**2])
    Q = np.diag([cfg.accel_std**2, cfg.omega_std**2])

    meas_ptr = 0
    for k in range(n):
        x = propagate(x, sim.imu_accel[k], sim.imu_omega[k], cfg.dt_imu)
        F, G = motion_jacobian(x, cfg.dt_imu, sim.imu_accel[k], sim.imu_omega[k])
        P = F @ P @ F.T + G @ Q @ G.T

        if meas_ptr < len(sim.pos_indices) and k == sim.pos_indices[meas_ptr]:
            H = np.array([[1.0, 0.0, 0.0, 0.0], [0.0, 1.0, 0.0, 0.0]])
            z = sim.pos_meas[meas_ptr]
            y = z - H @ x
            S = H @ P @ H.T + R
            K = P @ H.T @ np.linalg.inv(S)
            x = x + K @ y
            P = (np.eye(len(x)) - K @ H) @ P
            meas_ptr += 1
        x_est[k] = x
    return x_est


def iekf_filter(sim: SimData, cfg: SimConfig, max_iters: int = 5) -> np.ndarray:
    n = len(sim.t)
    x_est = np.zeros((n, 4))
    x = np.array([0.0, 0.0, 0.0, 0.5])
    P = np.diag([1.0, 1.0, (5 * math.pi / 180) ** 2, 1.0])
    R = np.diag([cfg.pos_std**2, cfg.pos_std**2])
    Q = np.diag([cfg.accel_std**2, cfg.omega_std**2])

    meas_ptr = 0
    for k in range(n):
        x = propagate(x, sim.imu_accel[k], sim.imu_omega[k], cfg.dt_imu)
        F, G = motion_jacobian(x, cfg.dt_imu, sim.imu_accel[k], sim.imu_omega[k])
        P = F @ P @ F.T + G @ Q @ G.T

        if meas_ptr < len(sim.pos_indices) and k == sim.pos_indices[meas_ptr]:
            z = sim.pos_meas[meas_ptr]
            x_iter = x.copy()
            P_iter = P.copy()
            H = np.array([[1.0, 0.0, 0.0, 0.0], [0.0, 1.0, 0.0, 0.0]])
            for _ in range(max_iters):
                y = z - H @ x_iter
                S = H @ P_iter @ H.T + R
                K = P_iter @ H.T @ np.linalg.inv(S)
                dx = K @ y
                x_iter = x_iter + dx
                P_iter = (np.eye(len(x)) - K @ H) @ P_iter
            x, P = x_iter, P_iter
            meas_ptr += 1
        x_est[k] = x
    return x_est


## Visualization helpers
Measurement markers are aligned with the ground-truth trajectory for clarity.


In [ ]:
def plot_compare(sim: SimData, x_ekf: np.ndarray, x_iekf: np.ndarray, title: str):
    plt.figure(figsize=(7, 6))
    plt.plot(sim.x_true[:, 0], sim.x_true[:, 1], label='Truth', lw=2)
    plt.plot(x_ekf[:, 0], x_ekf[:, 1], label='EKF', lw=2)
    plt.plot(x_iekf[:, 0], x_iekf[:, 1], label='IEKF', lw=2)
    plt.scatter(sim.x_true[sim.pos_indices, 0], sim.x_true[sim.pos_indices, 1],
                c='k', marker='x', label='Position measurement times')
    plt.axis('equal'); plt.grid(True)
    plt.xlabel('x [m]'); plt.ylabel('y [m]'); plt.title(title); plt.legend()

def rmse(a: np.ndarray, b: np.ndarray, fields=(0, 1)):
    diff = a[:, fields] - b[:, fields]
    return math.sqrt(np.mean(np.sum(diff**2, axis=1)))


## Baseline comparison
Run EKF and IEKF on the same dataset. Because the measurement is linear, both filters should be very similar, but IEKF is shown for completeness.


In [ ]:
cfg = SimConfig()
sim = simulate_truth_and_measurements(cfg)
x_ekf = ekf_filter(sim, cfg)
x_iekf = iekf_filter(sim, cfg)

plot_compare(sim, x_ekf, x_iekf, title='EKF vs IEKF (truth-aligned measurement markers)')
plt.show()

print('Baseline RMSE (pos): EKF =', rmse(x_ekf, sim.x_true), ' IEKF =', rmse(x_iekf, sim.x_true))


## Noise experiments
We sweep both IMU noise and position noise. The IEKF should not bring large gains for linear measurements, but we include the comparison to satisfy the assignment and to show any small differences.


In [ ]:
imu_levels = [(0.05, 0.01), (0.2, 0.03), (0.5, 0.1)]
    pos_levels = [0.2, 0.5, 1.5]

    imu_results = []
    for a_std, w_std in imu_levels:
        cfg_i = SimConfig(accel_std=a_std, omega_std=w_std)
        sim_i = simulate_truth_and_measurements(cfg_i)
        ekf_i = ekf_filter(sim_i, cfg_i)
        iekf_i = iekf_filter(sim_i, cfg_i)
        imu_results.append((a_std, w_std, rmse(ekf_i, sim_i.x_true), rmse(iekf_i, sim_i.x_true)))

    pos_results = []
    for p_std in pos_levels:
        cfg_p = SimConfig(pos_std=p_std)
        sim_p = simulate_truth_and_measurements(cfg_p)
        ekf_p = ekf_filter(sim_p, cfg_p)
        iekf_p = iekf_filter(sim_p, cfg_p)
        pos_results.append((p_std, rmse(ekf_p, sim_p.x_true), rmse(iekf_p, sim_p.x_true)))

    print('IMU noise sweep (accel_std, omega_std, EKF RMSE, IEKF RMSE):')
    for row in imu_results:
        print(row)
    print('
Position noise sweep (pos_std, EKF RMSE, IEKF RMSE):')
    for row in pos_results:
        print(row)

    fig, axs = plt.subplots(1, 2, figsize=(12, 4))
    axs[0].plot([r[0] for r in imu_results], [r[2] for r in imu_results], 'o-', label='EKF')
    axs[0].plot([r[0] for r in imu_results], [r[3] for r in imu_results], 's--', label='IEKF')
    axs[0].set_xlabel('Acceleration noise std [m/s^2]'); axs[0].set_ylabel('Position RMSE [m]')
    axs[0].set_title('Sensitivity to IMU noise'); axs[0].grid(True); axs[0].legend()

    axs[1].plot([r[0] for r in pos_results], [r[1] for r in pos_results], 'o-', label='EKF')
    axs[1].plot([r[0] for r in pos_results], [r[2] for r in pos_results], 's--', label='IEKF')
    axs[1].set_xlabel('Position noise std [m]'); axs[1].set_ylabel('Position RMSE [m]')
    axs[1].set_title('Sensitivity to position noise'); axs[1].grid(True); axs[1].legend()

    plt.tight_layout(); plt.show()


## Observations
* **Higher IMU noise** makes both filters rely on noisier propagation; errors between position updates increase. IEKF and EKF remain close because the measurement model is linear.
* **Higher position noise** weakens the correction; both filters lean on propagation and drift more, so RMSE rises.
* **IEKF vs EKF:** With linear position measurements, IEKF offers little advantage. If the measurement were nonlinear (e.g., range-bearing), IEKF iterations would improve linearization accuracy and could outperform EKF.
